# Comparing Year Prediction Using Complex Audio Features vs Song Lyrics

## Special Notes
1. We can write the paper in here and then hide the code when we knit it to PDF using the following command:
    (jupyter nbconvert --to pdf --template hidecode Example.ipynb
2. Bag of Words data is stored in a SQLite database that we will have to figure out how to use over spring break


In [1]:
import h5py
import numpy as np
import pandas as pd
import tables
import os
import sys
import time
import glob
import scipy.io as sio
import hdf5_getters
from pandas import read_hdf
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
#from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import GaussianNB

Using TensorFlow backend.


In [2]:
#################### THIS CREATES THE PANDAS DATAFRAME FROM THE DATAFILES WE HAVE ###########################

def get_all_files(basedir,ext='.h5') :
    """
    From a root directory, go through all subdirectories
    and find all files with the given extension.
    Return all absolute paths in a list.
    """
    allfiles = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files :
            allfiles.append( os.path.abspath(f) )
    return allfiles

def transfer(h5path,matpath=None,force=False):
    """
    Transfer an HDF5 song file (.h5) to a matfile (.mat)
    If there are more than one song in the HDF5 file, each
    field name gets a number happened: 1, 2, 3, ...., numfiles
    PARAM
        h5path  - path to the HDF5 song file
        matpath - path to the new matfile, same as HDF5 path
                  with a different extension by default
        force   - if True and matfile exists, overwrite
    RETURN
        True if the file was transfered, False if there was
        a problem.
        Could also raise an IOException
    NOTE
        All the data has to be loaded in memory! be careful
        if one file contains tons of songs!
    """
    # sanity checks
    if not os.path.isfile(h5path):
        print ('path to HF5 files does not exist:',h5path)
        return False
    if not os.path.splitext(h5path)[1] == '.h5':
        print ('expecting a .h5 extension for file:',h5path)
        return False
    # check matfile
    if matpath is None:
        matpath = os.path.splitext(h5path)[0] + '.mat'
    if os.path.exists(matpath):
        if not force: 
            print('matfile',matpath,'already exists (delete or force):')
            return False
    # get all getters! we assume that all we need is in hdf5_getters.py
    # further assume that they have the form get_blablabla and that's the
    # only thing that has that form
    getters = list(filter(lambda x: x[:4] == 'get_', hdf5_getters.__dict__.keys()))
    getters.remove("get_num_songs") # special case
    # open h5 file
    h5 = hdf5_getters.open_h5_file_read(h5path)
    # transfer
    nSongs = hdf5_getters.get_num_songs(h5)
    matdata = {'transfer_note':'transferred on '+time.ctime()+' from file: '+h5path}
    try:
        # iterate over songs
        for songidx in range(nSongs):
            # iterate over getter
            for getter in getters:
                gettername = getter[4:]
                if nSongs > 1:
                    gettername += str(songidx+1)
                data = hdf5_getters.__getattribute__(getter)(h5,songidx)
                matdata[gettername] = data
    except MemoryError:
        print('asdfasdfasdfasdf')
        raise
    finally:
        # close h5
        h5.close()
    # create
    return matdata



h5s = get_all_files('MillionSongSubset/data/')

df = pd.DataFrame()
for file in h5s:
    xd = transfer(file)
    df = df.append(pd.Series(xd), ignore_index=True)

df.head(10)

,analysis_sample_rate,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_mbtags,...,tatums_confidence,tatums_start,tempo,time_signature,time_signature_confidence,title,track_7digitalid,track_id,transfer_note,year
0,22050.0,165270.0,0.581794,0.401998,b'ARD7TVE1187B99BFB1',NaN,b'California - LA',NaN,b'e77e51a5-4761-45b3-9847-2051f811e366',[],...,"[0.779, 0.734, 0.674, 0.637, 0.597, 0.532, 0.4...","[0.28519, 0.58521, 0.89422, 1.19196, 1.49119, ...",92.198,4.0,0.778,"b""I Didn't Mean To""",3401791.0,b'TRAAAAW128F429D538',transferred on Sun Mar 10 12:24:29 2019 from f...,0.0
1,22050.0,1998.0,0.630630,0.417500,b'ARMJAGH1187FB546F3',35.14968,"b'Memphis, TN'",-90.04892,b'1c78ab62-db33-4433-8d0b-7c8dcf1849c2',[b'classic pop and rock'],...,"[0.969, 0.929, 0.897, 0.871, 0.856, 0.845, 0.8...","[0.20627, 0.45904, 0.71054, 0.96834, 1.21836, ...",121.274,4.0,0.384,b'Soul Deep',3400270.0,b'TRAAABD128F429CF47',transferred on Sun Mar 10 12:24:29 2019 from f...,1969.0
2,22050.0,290021.0,0.487357,0.343428,b'ARKRRTF1187B9984DA',NaN,b'',NaN,b'7a273984-edd9-4451-9c4d-39b38f05ebcd',[],...,"[0.482, 0.676, 0.627, 0.549, 0.279, 0.264, 0.2...","[0.42132, 0.73152, 1.06609, 1.39732, 1.72854, ...",100.070,1.0,0.000,b'Amor De Cabaret',5703798.0,b'TRAAADZ128F9348C2E',transferred on Sun Mar 10 12:24:30 2019 from f...,0.0
3,22050.0,19072.0,0.630382,0.454231,b'AR7G5I41187FB4CE6C',NaN,"b'London, England'",NaN,b'e188a520-9cb7-4f73-a3d7-2f70c6538e92',"[b'uk', b'british', b'english']",...,"[0.601, 0.556, 0.523, 0.49, 0.466, 0.44, 0.428...","[0.56254, 0.81002, 1.05749, 1.30621, 1.55494, ...",119.293,4.0,0.000,b'Something Girls',3226795.0,b'TRAAAEF128F4273421',transferred on Sun Mar 10 12:24:30 2019 from f...,1982.0
4,22050.0,30973.0,0.651046,0.401724,b'ARXR32B1187FB57099',NaN,b'',NaN,b'c6903a2e-063c-4f91-a284-17b8f421be7b',[],...,"[1.0, 0.98, 0.932, 0.87, 0.82, 0.793, 0.768, 0...","[0.13576, 0.36918, 0.59914, 0.83141, 1.06368, ...",129.738,4.0,0.562,b'Face the Ashes',6795666.0,b'TRAAAFD128F92F423A',transferred on Sun Mar 10 12:24:30 2019 from f...,2007.0
5,22050.0,432935.0,0.535293,0.385471,b'ARKFYS91187B98E58F',NaN,b'',NaN,b'79c403f9-5467-4f23-8426-9ca3fc60a115',[],...,"[0.136, 0.127, 0.113, 0.112, 0.104, 0.09, 0.07...","[0.53929, 0.74856, 0.95987, 1.17118, 1.38249, ...",147.782,3.0,0.454,b'The Moon And I (Ordinary Day Album Version)',444964.0,b'TRAAAMO128F1481E7F',transferred on Sun Mar 10 12:24:30 2019 from f...,0.0
6,22050.0,17970.0,0.556496,0.261941,b'ARD0S291187B9B7BF5',NaN,b'Ohio',NaN,b'56503d6d-094e-4c28-ae3d-04cc748ade5b',[],...,"[0.467, 0.474, 0.528, 0.541, 0.507, 0.482, 0.3...","[0.05611, 0.27253, 0.48785, 0.70535, 0.92722, ...",111.787,1.0,0.000,b'Keepin It Real (Skit)',276593.0,b'TRAAAMQ128F1460CD3',transferred on Sun Mar 10 12:24:30 2019 from f...,0.0
7,22050.0,21128.0,0.801136,0.605507,b'AR10USD1187B99F3F1',NaN,"b'Burlington, Ontario, Canada'",NaN,b'd89de379-665d-425c-b2e9-41b95d1edb36',[],...,"[0.292, 0.284, 0.282, 0.274, 0.27, 0.237, 0.21...","[0.36129, 0.65428, 0.94433, 1.24174, 1.53768, ...",101.430,3.0,0.408,b'Drop of Rain',90004.0,b'TRAAAPK128E0786D96',transferred on Sun Mar 10 12:24:30 2019 from f...,0.0
8,22050.0,276891.0,0.426668,0.332276,b'AR8ZCNI1187B9A069B',NaN,b'',NaN,b'19d232b9-b4d7-4dc8-b259-bf65efb655b1',[],...,"[0.121, 0.124, 0.126, 0.128, 0.13, 0.131, 0.13...","[1.22595, 1.39961, 1.57241, 1.74174, 1.91886, ...",86.643,4.0,0.487,b'Pink World',3996579.0,b'TRAAARJ128F9320760',transferred on Sun Mar 10 12:24:30 2019 from f...,1984.0
9,22050.0,242273.0,0.550514,0.422706,b'ARNTLGG11E2835DDB9',NaN,b'',NaN,b'4d96f7d0-2f0e-4e92-ba70-a405f96f8cec',[],...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.09933, 0.36057, 0.62445, 0.88967, 1.15423, ...",114.041,4.0,0.878,b'Insatiable (Instrumental Version)',7684249.0,b'TRAAAVG12903CFA543',transferred on Sun Mar 10 12:24:30 2019 from f...,0.0


In [3]:
############ HERE WE WILL CLEAN THE DATA ########################

# All songs have the same analysis_sample_rate so I remove it
newdf = df.drop("analysis_sample_rate",axis=1)

# Remove rows that have year==0 (no year) because we cannot use these data points
newdf = newdf[newdf.year != 0]

print(newdf.shape)
# !pip install ipython-cache
# import cache_magic
# %cache DF = newdf

(4680, 54)


In [4]:
workingData = df

In [5]:
workingData = workingData.drop(["artist_familiarity","artist_location","artist_mbtags", "artist_mbtags_count", "title"], axis=1)

workingData = workingData.drop(["artist_7digitalid", "artist_id", "artist_mbid", "artist_name", "artist_playmeid", "artist_terms", "artist_terms_freq"], axis=1)

workingData = workingData.drop(["artist_terms_weight", "audio_md5", "transfer_note", "similar_artists"], axis=1)

workingData = workingData.drop(["track_id", "track_7digitalid","song_id","release","release_7digitalid"], axis=1)

workingData.head(5)


,analysis_sample_rate,artist_hotttnesss,artist_latitude,artist_longitude,bars_confidence,bars_start,beats_confidence,beats_start,danceability,duration,...,segments_start,segments_timbre,song_hotttnesss,start_of_fade_out,tatums_confidence,tatums_start,tempo,time_signature,time_signature_confidence,year
0,22050.0,0.401998,NaN,NaN,"[0.643, 0.746, 0.722, 0.095, 0.091, 0.362, 0.4...","[0.58521, 2.94247, 5.14371, 7.74554, 10.36149,...","[0.834, 0.851, 0.65, 0.635, 0.532, 0.753, 0.62...","[0.58521, 1.19196, 1.78893, 2.37813, 2.94247, ...",0.0,218.93179,...,"[0.0, 0.24671, 0.47116, 0.80376, 0.89551, 1.12...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,...",0.602120,218.932,"[0.779, 0.734, 0.674, 0.637, 0.597, 0.532, 0.4...","[0.28519, 0.58521, 0.89422, 1.19196, 1.49119, ...",92.198,4.0,0.778,0.0
1,22050.0,0.417500,35.14968,-90.04892,"[0.007, 0.259, 0.172, 0.404, 0.011, 0.016, 0.0...","[0.71054, 2.71502, 4.70861, 6.69288, 8.66941, ...","[1.0, 0.945, 0.714, 0.973, 0.818, 0.974, 0.878...","[0.20627, 0.71054, 1.21836, 1.71841, 2.21729, ...",0.0,148.03546,...,"[0.0, 0.14803, 0.68104, 0.95492, 1.19878, 1.45...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,...",NaN,137.915,"[0.969, 0.929, 0.897, 0.871, 0.856, 0.845, 0.8...","[0.20627, 0.45904, 0.71054, 0.96834, 1.21836, ...",121.274,4.0,0.384,1969.0
2,22050.0,0.343428,NaN,NaN,"[0.98, 0.399, 0.185, 0.27, 0.422, 0.0, 0.445, ...","[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....","[0.98, 0.399, 0.185, 0.27, 0.422, 0.0, 0.445, ...","[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....",0.0,177.47546,...,"[0.0, 0.28154, 0.48395, 0.6937, 0.97859, 1.361...","[[0.0, 171.124, 9.459, -28.489, 57.482, -50.06...",NaN,172.304,"[0.482, 0.676, 0.627, 0.549, 0.279, 0.264, 0.2...","[0.42132, 0.73152, 1.06609, 1.39732, 1.72854, ...",100.070,1.0,0.000,0.0
3,22050.0,0.454231,NaN,NaN,"[0.017, 0.05, 0.014, 0.008, 0.114, 0.019, 0.08...","[1.30621, 3.29887, 5.30252, 7.32327, 9.33775, ...","[0.809, 0.616, 0.789, 0.66, 0.439, 0.758, 0.60...","[0.81002, 1.30621, 1.80617, 2.2996, 2.80049, 3...",0.0,233.40363,...,"[0.0, 0.70517, 1.03052, 1.21052, 1.52404, 1.72...","[[24.937, 37.465, 177.22, -216.443, 56.3, 202....",NaN,217.124,"[0.601, 0.556, 0.523, 0.49, 0.466, 0.44, 0.428...","[0.56254, 0.81002, 1.05749, 1.30621, 1.55494, ...",119.293,4.0,0.000,1982.0
4,22050.0,0.401724,NaN,NaN,"[0.175, 0.409, 0.639, 0.067, 0.016, 0.066, 0.0...","[1.06368, 2.91491, 4.76729, 6.61852, 8.46978, ...","[0.883, 0.738, 0.484, 0.609, 0.625, 0.719, 0.4...","[0.13576, 0.59914, 1.06368, 1.52591, 1.99045, ...",0.0,209.60608,...,"[0.0, 0.06603, 0.24395, 0.57034, 0.92567, 1.26...","[[0.089, 169.621, 5.435, -30.061, 54.144, -50....",0.604501,198.699,"[1.0, 0.98, 0.932, 0.87, 0.82, 0.793, 0.768, 0...","[0.13576, 0.36918, 0.59914, 0.83141, 1.06368, ...",129.738,4.0,0.562,2007.0


In [6]:
######### REMOVING CONFIDENCE MEASURES ####################
workingData = workingData.drop(["bars_confidence", "beats_confidence","key_confidence","mode_confidence","sections_confidence", "segments_confidence", "tatums_confidence", "time_signature_confidence"], axis=1)
workingData.head(5)



,analysis_sample_rate,artist_hotttnesss,artist_latitude,artist_longitude,bars_start,beats_start,danceability,duration,end_of_fade_in,energy,...,segments_loudness_start,segments_pitches,segments_start,segments_timbre,song_hotttnesss,start_of_fade_out,tatums_start,tempo,time_signature,year
0,22050.0,0.401998,NaN,NaN,"[0.58521, 2.94247, 5.14371, 7.74554, 10.36149,...","[0.58521, 1.19196, 1.78893, 2.37813, 2.94247, ...",0.0,218.93179,0.247,0.0,...,"[-60.0, -60.0, -40.84, -40.401, -38.456, -39.6...","[[0.946, 0.684, 0.679, 0.941, 0.744, 0.633, 0....","[0.0, 0.24671, 0.47116, 0.80376, 0.89551, 1.12...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,...",0.602120,218.932,"[0.28519, 0.58521, 0.89422, 1.19196, 1.49119, ...",92.198,4.0,0.0
1,22050.0,0.417500,35.14968,-90.04892,"[0.71054, 2.71502, 4.70861, 6.69288, 8.66941, ...","[0.20627, 0.71054, 1.21836, 1.71841, 2.21729, ...",0.0,148.03546,0.148,0.0,...,"[-60.0, -60.0, -23.521, -25.16, -27.133, -24.2...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[0.0, 0.14803, 0.68104, 0.95492, 1.19878, 1.45...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,...",NaN,137.915,"[0.20627, 0.45904, 0.71054, 0.96834, 1.21836, ...",121.274,4.0,1969.0
2,22050.0,0.343428,NaN,NaN,"[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....","[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....",0.0,177.47546,0.282,0.0,...,"[-60.0, -59.9, -12.744, -12.003, -12.991, -15....","[[1.0, 0.911, 0.18, 0.334, 0.327, 0.344, 0.302...","[0.0, 0.28154, 0.48395, 0.6937, 0.97859, 1.361...","[[0.0, 171.124, 9.459, -28.489, 57.482, -50.06...",NaN,172.304,"[0.42132, 0.73152, 1.06609, 1.39732, 1.72854, ...",100.070,1.0,0.0
3,22050.0,0.454231,NaN,NaN,"[1.30621, 3.29887, 5.30252, 7.32327, 9.33775, ...","[0.81002, 1.30621, 1.80617, 2.2996, 2.80049, 3...",0.0,233.40363,0.000,0.0,...,"[-60.0, -27.665, -21.241, -15.222, -18.915, -1...","[[0.651, 0.592, 0.647, 0.494, 0.683, 0.919, 1....","[0.0, 0.70517, 1.03052, 1.21052, 1.52404, 1.72...","[[24.937, 37.465, 177.22, -216.443, 56.3, 202....",NaN,217.124,"[0.56254, 0.81002, 1.05749, 1.30621, 1.55494, ...",119.293,4.0,1982.0
4,22050.0,0.401724,NaN,NaN,"[1.06368, 2.91491, 4.76729, 6.61852, 8.46978, ...","[0.13576, 0.59914, 1.06368, 1.52591, 1.99045, ...",0.0,209.60608,0.066,0.0,...,"[-60.0, -59.828, -19.551, -32.609, -21.899, -2...","[[1.0, 0.529, 0.407, 0.423, 0.524, 0.509, 0.65...","[0.0, 0.06603, 0.24395, 0.57034, 0.92567, 1.26...","[[0.089, 169.621, 5.435, -30.061, 54.144, -50....",0.604501,198.699,"[0.13576, 0.36918, 0.59914, 0.83141, 1.06368, ...",129.738,4.0,2007.0


In [7]:
############### bars_start, beats_start, sections_start, segments_loudness_max, 
############### segments_loudness_max_time, segments_loudness_max_start, segments_start, tatums_start
############### these features are all in array form


############### segments_pitches, segments_timbre 
############### are in 2d array form


############ Calculate weighted average for segments_timbre and segments_pitches

#workingData.loc[2,"segments_timbre"]
# workingData.to_csv("WorkingData.csv", ",")
# import pandas as pd

# workingData = pd.read_csv("WorkingData.csv", " ")
# print("done")
# sworkingData["segments_start"].iloc[s].split(" ")
# workingData["segments_pitches"].iloc[s].split(" ")

addingNewTimber =  pd.DataFrame(columns=['Tim1','Tim2','Tim3','Tim4','Tim5','Tim6','Tim7',
                                          'Tim8','Tim9','Tim10','Tim11','Tim12'])
addingNewPitch =  pd.DataFrame(columns=['Pitch1','Pitch2','Pitch3','Pitch4','Pitch5','Pitch6','Pitch7',
                                          'Pitch8','Pitch9','Pitch10','Pitch11','Pitch12'])

for s in range(workingData.shape[0]):
    valLengthsT = []
    valLengthsP = []
    storeT = workingData["segments_start"].iloc[s]
    storeP = workingData["segments_pitches"].iloc[s]
    for k in range(len(storeT)-1):
        valLengthsT.append(storeT[k+1]-storeT[k+1])
        valLengthsP.append(storeP[k+1]-storeP[k+1])
    valLengthsT.append(workingData["duration"].iloc[s]-storeT[-1])
    valLengthsP.append(workingData["duration"].iloc[s]-storeP[-1])
    myWeightedMeanT = np.repeat(0,12).astype(float)
    myWeightedMeanP = np.repeat(0,12).astype(float)
    for q in range(workingData["segments_timbre"].iloc[s].shape[0]):
        myWeightedMeanT += valLengthsT[q]*(workingData["segments_timbre"].iloc[s])[q]
        myWeightedMeanP += valLengthsP[q]*(workingData["segments_timbre"].iloc[s])[q]
    myWeightedMeanT = myWeightedMeanT/workingData["duration"].iloc[s]
    myWeightedMeanP = myWeightedMeanP/workingData["duration"].iloc[s]
    
    addingNewTimber.loc[s] = myWeightedMeanT
    addingNewPitch.loc[s] = myWeightedMeanP

#workingData.append(addingNewVecInfo)
mergeTim = pd.concat([workingData, addingNewTimber], axis=1, sort=False)
finalWorkingDF = pd.concat([mergeTim, addingNewPitch], axis=1, sort=False)
finalWorkingDF.to_csv("FinalDF.csv", ",")
finalWorkingDF.head(5)

,analysis_sample_rate,artist_hotttnesss,artist_latitude,artist_longitude,bars_start,beats_start,danceability,duration,end_of_fade_in,energy,...,Pitch3,Pitch4,Pitch5,Pitch6,Pitch7,Pitch8,Pitch9,Pitch10,Pitch11,Pitch12
0,22050.0,0.401998,NaN,NaN,"[0.58521, 2.94247, 5.14371, 7.74554, 10.36149,...","[0.58521, 1.19196, 1.78893, 2.37813, 2.94247, ...",0.0,218.93179,0.247,0.0,...,-79.841478,8.863259,-53.759702,-26.742869,-5.537102,-92.185639,22.108005,19.302711,-21.781842,16.926603
1,22050.0,0.417500,35.14968,-90.04892,"[0.71054, 2.71502, 4.70861, 6.69288, 8.66941, ...","[0.20627, 0.71054, 1.21836, 1.71841, 2.21729, ...",0.0,148.03546,0.148,0.0,...,-6.873309,20.289421,43.542628,-50.036787,4.912887,45.033462,-27.173885,-6.312071,4.703293,-6.423300
2,22050.0,0.343428,NaN,NaN,"[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....","[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....",0.0,177.47546,0.282,0.0,...,-99.419082,347.182291,-5.632113,-10.669601,-73.693238,95.388630,-8.168712,-43.155660,-137.423790,5.076842
3,22050.0,0.454231,NaN,NaN,"[1.30621, 3.29887, 5.30252, 7.32327, 9.33775, ...","[0.81002, 1.30621, 1.80617, 2.2996, 2.80049, 3...",0.0,233.40363,0.000,0.0,...,51.774223,11.929518,9.978057,-13.800586,13.658217,20.780797,-73.369905,-11.486943,-22.934629,-34.280749
4,22050.0,0.401724,NaN,NaN,"[1.06368, 2.91491, 4.76729, 6.61852, 8.46978, ...","[0.13576, 0.59914, 1.06368, 1.52591, 1.99045, ...",0.0,209.60608,0.066,0.0,...,43.006366,169.472477,15.266810,-45.114801,-50.116896,86.399747,-17.554204,-32.974927,-35.801193,-5.641859


In [8]:
finalWorkingDF = finalWorkingDF.drop(["artist_latitude", "artist_longitude", "segments_timbre", "segments_start"], axis=1)
finalWorkingDF = finalWorkingDF.drop(["bars_start", "beats_start"], axis=1)

In [9]:
list(finalWorkingDF.columns.values)

['analysis_sample_rate',
 'artist_hotttnesss',
 'danceability',
 'duration',
 'end_of_fade_in',
 'energy',
 'key',
 'loudness',
 'mode',
 'sections_start',
 'segments_loudness_max',
 'segments_loudness_max_time',
 'segments_loudness_start',
 'segments_pitches',
 'song_hotttnesss',
 'start_of_fade_out',
 'tatums_start',
 'tempo',
 'time_signature',
 'year',
 'Tim1',
 'Tim2',
 'Tim3',
 'Tim4',
 'Tim5',
 'Tim6',
 'Tim7',
 'Tim8',
 'Tim9',
 'Tim10',
 'Tim11',
 'Tim12',
 'Pitch1',
 'Pitch2',
 'Pitch3',
 'Pitch4',
 'Pitch5',
 'Pitch6',
 'Pitch7',
 'Pitch8',
 'Pitch9',
 'Pitch10',
 'Pitch11',
 'Pitch12']

In [10]:
finalWorkingDF = finalWorkingDF.drop(['sections_start','segments_pitches'], axis=1)
list(finalWorkingDF.columns.values)

['analysis_sample_rate',
 'artist_hotttnesss',
 'danceability',
 'duration',
 'end_of_fade_in',
 'energy',
 'key',
 'loudness',
 'mode',
 'segments_loudness_max',
 'segments_loudness_max_time',
 'segments_loudness_start',
 'song_hotttnesss',
 'start_of_fade_out',
 'tatums_start',
 'tempo',
 'time_signature',
 'year',
 'Tim1',
 'Tim2',
 'Tim3',
 'Tim4',
 'Tim5',
 'Tim6',
 'Tim7',
 'Tim8',
 'Tim9',
 'Tim10',
 'Tim11',
 'Tim12',
 'Pitch1',
 'Pitch2',
 'Pitch3',
 'Pitch4',
 'Pitch5',
 'Pitch6',
 'Pitch7',
 'Pitch8',
 'Pitch9',
 'Pitch10',
 'Pitch11',
 'Pitch12']

In [11]:
finalWorkingDF = finalWorkingDF.drop(['segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start'], axis=1)

In [12]:
finalWorkingDF = finalWorkingDF.drop('analysis_sample_rate', axis=1)
finalWorkingDF.iloc[:,5:20]
finalWorkingDF.to_pickle("finalWorkingDFwtatums")

In [13]:
tatums_starts = finalWorkingDF['tatums_start']
diffLists = []
for tatum_start in tatums_starts:
    diff = []
    old = 0.0
    for tatum in tatum_start:
        diff.append(tatum - old)
        old = tatum
    if len(tatum_start) == 0:
        diffLists.append(np.array([0.0]))
    else:
        diffLists.append(np.mean(np.array(diff)))
    
finalWorkingDF['tatums_mean'] = diffLists

In [14]:
finalWorkingDF = finalWorkingDF.drop('tatums_start', axis=1)
finalWorkingDF.to_pickle("finalWorkingDF")

In [21]:

finalWorkingDF = pd.read_pickle("finalWorkingDF")
finalWorkingDF.fillna(finalWorkingDF.mean(), inplace=True)
# Separatee data into training and testing set
y = finalWorkingDF['year']
x = finalWorkingDF.drop('year', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [22]:
x_train.to_pickle("XTrain")
y_train.to_pickle("YTrain")
x_test.to_pickle("XTest")
y_test.to_pickle("YTest")

In [28]:
x_trainNumPy = x_train.to_numpy()
x_testNumPy = x_test.to_numpy()
y_trainNumPy = y_train.to_numpy()
y_testNumPy = y_test.to_numpy()

In [36]:
###################### KNN CLASSIFIER ########################
from sklearn.metrics import mean_squared_error
n_neighbors = 15

nnClassifier = neighbors.KNeighborsClassifier(n_neighbors)
nnClassifier.fit(x_trainNumPy, y_trainNumPy)

# Get training and test predictions
predictedtrainyears = nnClassifier.predict(x_trainNumPy)
trainMSE = mean_squared_error(y_trainNumPy, predictedtrainyears)
testMSE = mean_squared_error(y_testNumPy, nnClassifier.predict(x_testNumPy))
# classifier_trainMSE = np.mean((nnClassifier.predict(x_trainNumPy) - y_trainNumPy)**2)
# classifier_testMSE = np.mean((nnClassifier.predict(x_testNumPy) - y_testNumPy)**2)

print("training MSE", trainMSE)
print("testing MSE", testMSE)

[0.]
training MSE 1866826.2995
testing MSE 1837225.9715


In [44]:
print(y)

0          0.0
1       1969.0
2          0.0
3       1982.0
4       2007.0
5          0.0
6          0.0
7          0.0
8       1984.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14      1986.0
15      1987.0
16         0.0
17         0.0
18      2004.0
19         0.0
20         0.0
21         0.0
22         0.0
23      2004.0
24      1985.0
25         0.0
26         0.0
27      2005.0
28      1997.0
29      1961.0
         ...  
9970    1985.0
9971       0.0
9972       0.0
9973       0.0
9974    2003.0
9975       0.0
9976       0.0
9977       0.0
9978    2003.0
9979       0.0
9980       0.0
9981    2009.0
9982    2000.0
9983    1996.0
9984    1978.0
9985       0.0
9986       0.0
9987       0.0
9988    2006.0
9989    2006.0
9990       0.0
9991    1996.0
9992       0.0
9993       0.0
9994    1989.0
9995    1998.0
9996    1998.0
9997       0.0
9998       0.0
9999    2005.0
Name: year, Length: 10000, dtype: float64


In [35]:
print(x_trainNumPy.shape)
print(x_testNumPy.shape)

(8000, 37)
(2000, 37)


In [ ]:
###################### KNN REGRESSION ########################
n_neighbors = 15

nnRegressor = neighbors.KNeighborsRegressor(n_neighbors)
nnRegressor.fit(x_train, y_train)

# Get training and test predictions
regressor_trainMSE = np.mean((nnRegressor.predict(x_train) - y_train)**2)
regressor_testMSE = np.mean((nnRegressor.predict(x_test) - y_test)**2)

print("training MSE", regressor_trainMSE)
print("testing MSE", regressor_testMSE)

In [ ]:
!pip install keras

In [ ]:
###################### NEURAL NETWORK ########################


# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(x_train, y_train, epochs=150, batch_size=10)

# evaluate the model
train_scores = model.evaluate(x_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], train_scores[1]*100))
test_scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], test_scores[1]*100))